In [1]:
import duckdb
import pandas as pd
import config
import json
import sqlite3

In [6]:
# db = duckdb.connect("../data/peeringdb.db", read_only=False)
db = sqlite3.connect("../data/peeringdb.db")
# latest peeringdb dump
input_file = config.BASE_DATA_DIR / 'pdb' / 'peeringdb_2_dump_2024_04_10.json'

In [7]:
# functions 
def get_json_data(filename):
    with open(filename) as f:
        data = json.load(f)
    return data


def get_object(filename, obj_type):
    data = get_json_data(filename)
    return data[obj_type]['data']

In [4]:
# check the keys in the json file
d = get_json_data(input_file)
d.keys()

dict_keys(['ixlan', 'ixfac', 'carrierfac', 'netixlan', 'ix', 'net', 'netfac', 'poc', 'api', 'fac', 'carrier', 'org', 'ixpfx', 'as_set', 'campus'])

In [16]:
# check the keys in the net object
net = get_object(input_file, 'net')
df = pd.DataFrame(net)
# drop social_media, notes, looking_glass, info_types, status_dashboard, poc_updated, policy_url, route_server
df.drop(columns=['social_media', 'notes', 'looking_glass', 'info_types', 'status_dashboard', 'poc_updated', 'policy_url', 'route_server'], inplace=True)

df.head(n=2)

,status,fac_count,netixlan_updated,info_ratio,id,policy_ratio,info_unicast,rir_status,created,name_long,...,ix_count,org_id,info_never_via_route_servers,info_type,policy_contracts,info_prefixes6,aka,info_prefixes4,info_ipv6,irr_as_set
0,ok,0,2021-09-22T00:06:59Z,,1,True,True,ok,2004-07-28T00:00:00Z,,...,0,8897,False,NSP,Required,10000,Formerly known as nLayer Communications,200000,True,
1,ok,195,2024-04-08T20:07:39Z,Heavy Outbound,2,False,True,ok,2004-07-28T00:00:00Z,,...,206,14,False,Content,Not Required,5000,,12000,True,AS-AKAMAI


In [15]:
def import_pdb_file(f, peering_db_engine=db):

    try:
        # record_date = '2023-10-09' # extract_date(f)
        json_data = get_json_data(f)
        print(json_data.keys())
        # get IX LAN data
        ix_lan = pd.DataFrame(json_data['ixlan']['data'])
        print("IX LAN data")
        # ix_lan['record_date'] = record_date
        # get IX data
        ix = pd.DataFrame(json_data['ix']['data'])
        ix = ix.drop(columns=['social_media'], errors='ignore')
        # ix['record_date'] = record_date
        # get network data
        net = pd.DataFrame(json_data['net']['data'])
        net = net.drop(columns=['social_media','aka', 'status_dashboard','looking_glass', 'website', 'name_long', 'route_server','notes','irr_as_set', 'info_types'], errors='ignore')
        print("Network data")
        # net['record_date'] = record_date
        # get network ix lan data
        ix_pres = pd.DataFrame(json_data['netixlan']['data'])
        print("Network IX LAN data")
        # ix_pres['record_date'] = record_date
        # get network facility data
        fac_p = pd.DataFrame(json_data['netfac']['data'])
        # fac_p['record_date'] = record_date
        print("Network facility data")
        # get facility data
        fac = pd.DataFrame(json_data['fac']['data'])
        fac.drop(columns=['website', 'notes', 'version', 'address1',
                          'address2', 'city', 'state', 'zipcode','social_media', 'aka', 'suite', 'floor','require_2fa',
                          'available_voltage_services', 'aka', 'property','name_long'], errors='ignore', inplace=True)
        print("Facility data", len(fac))
        # fac['record_date'] = record_date
        # get organization data
        org = pd.DataFrame(json_data['org']['data'])
        print("Organization data",len(org))
        # org['record_date'] = record_date
        org = org.drop(columns=['website', 'notes', 'version', 'address1',
                                'address2', 'city', 'state', 'zipcode','social_media', 'aka', 'suite', 'floor','require_2fa'], errors='ignore')
        # save all data to db
        ix_lan.to_sql('ix_lan', con=peering_db_engine, index=False, if_exists='replace')
        print("IX LAN data Inserted")
        ix.to_sql('ix', con=peering_db_engine, index=False, if_exists='replace')
        net.to_sql('net', con=peering_db_engine, index=False, if_exists='replace')
        print("Network data Inserted")
        ix_pres.to_sql('netixlan', con=peering_db_engine, index=False, if_exists='replace')
        print("Network IX LAN data Inserted")
        fac_p.to_sql('netfac', con=peering_db_engine, index=False, if_exists='replace')
        print("Network facility data Inserted")
        fac.to_sql('fac', con=peering_db_engine, index=False, if_exists='replace')
        print("Facility data Inserted")
        org.to_sql('org', con=peering_db_engine, index=False, if_exists='replace')
        print("Organization data Inserted")
        print(f"[+] Done processing {f.stem}")
    except Exception as e:
        print(f"[-] Error processing {f.stem}: {e}")

import_pdb_file(input_file)

dict_keys(['ixlan', 'ixfac', 'carrierfac', 'netixlan', 'ix', 'net', 'netfac', 'poc', 'api', 'fac', 'carrier', 'org', 'ixpfx', 'as_set', 'campus'])
IX LAN data
IX data
Network data
Network IX LAN data
Network facility data
Facility data 5366
Organization data 27994
IX LAN data Inserted
Network data Inserted
Network IX LAN data Inserted
Network facility data Inserted
Facility data Inserted
Organization data Inserted
[+] Done processing peeringdb_2_dump_2024_04_10
